In [1]:
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine

In [2]:
DATABASE = {
    'drivername': 'postgresql',
    'host': 'localhost',
    'port': '5432',
    'username': 'postgres',
    'password': '251016',
    'database': 'Biblioteca'
}

### Emprestimo e Devolução - Bibliotecario

In [3]:
import panel as pn

# Ativando a extensão Panel no Jupyter
pn.extension()

# Criando o menu principal
menu = pn.Column(
    pn.pane.Markdown("# Sistema de Gerenciamento de Biblioteca"),
    pn.pane.Markdown("## Escolha uma opção abaixo:"),
    pn.widgets.Button(name="Gerenciar Empréstimos e Devoluções", button_type="primary"),
    pn.widgets.Button(name="Realizar Reservas", button_type="primary"),
    pn.widgets.Button(name="Pagar Multas", button_type="primary"),
)

menu.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'831a3eeb-e601-41fe-a638-08ccacf27294': {'version…

In [4]:
pn.extension()

# Widgets para capturar dados de usuário e livro
usuario_id = pn.widgets.IntInput(name="ID do Usuário", value=0)
livro_id = pn.widgets.IntInput(name="ID do Livro", value=0)
button_emprestimo = pn.widgets.Button(name="Realizar Empréstimo", button_type="primary")
button_devolucao = pn.widgets.Button(name="Registrar Devolução", button_type="success")

# Funções para realizar operações no banco de dados
def realizar_emprestimo(event):
    try:
        data_emprestimo = datetime.now()
        data_devolucao = data_emprestimo + timedelta(days=30)
        query = f"INSERT INTO emprestimo (id_usuario, id_livro, data_emprestimo, data_devolucao, status) VALUES ({usuario_id.value}, {livro_id.value}, '{data_emprestimo}', '{data_devolucao}', 'Ativo')"
        engine.execute(query)
        pn.state.notifications.success('Empréstimo realizado com sucesso!')
    except Exception as e:
        pn.state.notifications.error('Erro ao realizar empréstimo: ' + str(e))

def registrar_devolucao(event):
    try:
        query = f"UPDATE emprestimo SET status = 'Concluído', data_devolucao = '{datetime.now()}' WHERE id_usuario = {usuario_id.value} AND id_livro = {livro_id.value} AND status = 'Ativo'"
        engine.execute(query)
        pn.state.notifications.success('Devolução registrada com sucesso!')
    except Exception as e:
        pn.state.notifications.error('Erro ao registrar devolução: ' + str(e))

# Associando funções aos botões
button_emprestimo.on_click(realizar_emprestimo)
button_devolucao.on_click(registrar_devolucao)

# Interface para o bibliotecário
emprestimos_devolucoes = pn.Column(
    pn.pane.Markdown("# Gestão de Empréstimos e Devoluções"),
    usuario_id, livro_id, button_emprestimo, button_devolucao
)

emprestimos_devolucoes.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'fddb2153-991a-4c0e-9052-d1fc8f6d16ac': {'version…

### Reserva de Livro - Aluno/  Professor

In [5]:
import panel as pn
from datetime import datetime, timedelta

# Widgets para capturar dados do usuário e livro
usuario_id = pn.widgets.IntInput(name="ID do Usuário", value=0)
livro_id = pn.widgets.IntInput(name="ID do Livro", value=0)
button_reserva = pn.widgets.Button(name="Reservar Livro", button_type="primary")

# Função para realizar reserva
def realizar_reserva(event):
    try:
        data_reserva = datetime.now()
        query = f"INSERT INTO reserva (id_usuario, id_livro, data_reserva, status) VALUES ({usuario_id.value}, {livro_id.value}, '{data_reserva}', 'Ativa')"
        engine.execute(query)
        pn.state.notifications.success('Reserva realizada com sucesso!')
    except Exception as e:
        pn.state.notifications.error('Erro ao realizar reserva: ' + str(e))

button_reserva.on_click(realizar_reserva)

# Interface para reserva de livros
reservas = pn.Column(
    pn.pane.Markdown("# Reserva de Livros"),
    usuario_id, livro_id, button_reserva
)

reservas.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'61bbae9e-0608-4760-96f4-35cd089d30ab': {'version…

### Pagamento de Multas

In [6]:
pn.extension()

# Widgets
usuario_id = pn.widgets.IntInput(name="ID do Usuário", value=0, placeholder="Insira o ID do Usuário")
button_consulta_multa = pn.widgets.Button(name="Consultar Multas", button_type="primary")
metodo_pagamento = pn.widgets.RadioButtonGroup(name="Método de Pagamento", options=['GRU', 'Pix', 'Cartão'])
button_pagar = pn.widgets.Button(name="Pagar Multa", button_type="success")

# Funções
def consultar_multas(event):
    try:
        query = f"SELECT * FROM Multa WHERE ID_Usuario = {usuario_id.value}"
        df = pd.read_sql_query(query, con)
        table = pn.widgets.Tabulator(df, layout='fit_data')
        return table
    except Exception as e:
        return pn.pane.Alert('Erro ao consultar multas!')

def pagar_multa(event):
    try:
        metodo = metodo_pagamento.value
        pn.state.notifications.success(f'Multa paga com sucesso via {metodo}!')
    except Exception as e:
        pn.state.notifications.error('Erro ao processar pagamento!')

button_consulta_multa.on_click(consultar_multas)
button_pagar.on_click(pagar_multa)

# Interface
pn.Column(
    pn.pane.Markdown("## Pagamento de Multas"),
    usuario_id, button_consulta_multa, metodo_pagamento, button_pagar
).servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'8d48d8ea-15ea-4a1c-90b2-c5291a5e5f7f': {'version…